In [100]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import re

In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [102]:
train = pd.read_csv("train_news.csv")
test = pd.read_csv("test_news.csv")
submit = pd.read_csv("base_submission_news.csv")

display(train.head())
display(test.head())

,content,topic
0,Фото: Михаил Воскресенский / РИА Новости Марин...,1
1,Фото: Mike Segar / Reuters Мария Сметанина Руб...,1
2,Фото: Алексей Даничев / РИА Новости Варвара Ко...,0
3,Фото: Юрий Абрамочкин / РИА Новости Дмитрий Ок...,0
4,Роман Головченко Фото: vpk.gov.by / Globallook...,3


,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...


In [103]:
df_train = train.copy()
df_test = test.copy()

display(df_test)

,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...
...,...
26270,Фото: РИА Новости Алевтина Запольская Главное ...
26271,Вадим Гутцайт Фото: Sergei CHUZAVKOV / Europea...
26272,Фото: Олег Харсеев / Коммерсантъ Александр Кур...
26273,Владимир Зеленский Фото: Yves Herman / Reuters...


In [105]:
from pandas.api.types import (
    is_object_dtype,
    is_integer_dtype,
    is_float_dtype,
    is_bool_dtype,
    is_datetime64_any_dtype,
)
# словарь для хранения типов колонок
types = {
    "numeric_integer": [],
    "numeric_float": [],
    "binary": [],
    "categorical": [],
    "datetime": [],
    "text": []
}

# определяю типы данных по каждому столбцу
for col in df_train.columns:
    col_data = df_train[col]
    col_type = col_data.dtype

    if is_integer_dtype(col_data):
      types["numeric_integer"].append(col)

    if is_float_dtype(col_data):
      types["numeric_float"].append(col)

    if is_bool_dtype(col_data):
      types["binary"].append(col)

    if is_object_dtype(col_data):
      types["text"].append(col)

    if is_datetime64_any_dtype(col_data):
      types["datetime"].append(col)

numeric_cols = (
    types["numeric_integer"] + types["numeric_float"]
)

def get_text_stat(df, text_cols):
    if len(types["text"]) > 0:
        text_desc = pd.DataFrame(index=types["text"])
        for col in types["text"]:
            col_data = df[col].dropna().astype(str)

            lengths = col_data.apply(len)

            text_desc.loc[col, "mean_len"] = lengths.mean()
            text_desc.loc[col, "median_len"] = lengths.median()
            text_desc.loc[col, "max_len"] = lengths.max()
            text_desc.loc[col, "min_len"] = lengths.min()
            text_desc.loc[col, "missing_pct"] = df[col].isna().mean()
            text_desc.loc[col, "unique_cnt"] = col_data.nunique()
            
    return text_desc

print("Статистика по текстовым данным для df_train")
display(get_text_stat(df_train, types['text']))
print("Статистика по текстовым данным для df_test")
display(get_text_stat(df_test, types['text']))

Статистика по текстовым данным для df_train


,mean_len,median_len,max_len,min_len,missing_pct,unique_cnt
content,1599.546214,1179.0,43128.0,72.0,0.04007,19514.0


Статистика по текстовым данным для df_test


,mean_len,median_len,max_len,min_len,missing_pct,unique_cnt
content,1669.993073,1152.0,145399.0,14.0,0.0,26274.0


In [106]:
df_train[df_train.isna().any(axis=1)]

,content,topic
2313,NaN,1
3723,NaN,0
3981,NaN,0
4416,NaN,0
5329,NaN,2
...,...,...
19348,NaN,7
19368,NaN,7
19505,NaN,7
19629,NaN,7


In [107]:
# Сколько строк было
print("До:", df_train.shape)

# Дропаем все строки с NaN
df_train = df_train.dropna().reset_index(drop=True)

# Сколько осталось
print("После:", df_train.shape)

До: (21088, 2)
После: (20243, 2)


In [108]:
dups_mask = df_train["content"].duplicated(keep=False)
df_dups = df_train[dups_mask].sort_values("content")

display(df_dups)

,content,topic
133,Крушение императорского поезда около станции ...,8
10874,Крушение императорского поезда около станции ...,8
1886,Москвичи на церемонии торжественного захороне...,8
12338,Москвичи на церемонии торжественного захороне...,8
13207,Обстановка в мире накалена до предела. Впервы...,8
...,...,...
11573,Фото: 力力摄影日记 / Unsplash Елена Апазиди Астроном...,8
14274,Харри Кейн Фото: Ciro De Luca / Reuters Ксения...,4
8363,Харри Кейн Фото: Ciro De Luca / Reuters Ксения...,4
498,фото: Екатерина Якель / Коммерсантъ Валентина ...,7


In [109]:
print("До:", df_train.shape)

df_train = df_train.drop_duplicates(subset=["content"]).reset_index(drop=True)

print("После:", df_train.shape)

До: (20243, 2)
После: (19514, 2)


In [110]:
df_train["topic"].value_counts(normalize=True)

topic
7    0.190632
8    0.185303
1    0.172184
4    0.152711
2    0.088296
0    0.078354
3    0.065389
5    0.045967
6    0.021164
Name: proportion, dtype: float64

In [111]:
df_train["content_len"] = df_train["content"].str.len()
df_train["content_len"].describe()

count    19514.000000
mean      1592.834222
std       2180.506323
min         72.000000
25%        931.000000
50%       1174.000000
75%       1520.000000
max      43128.000000
Name: content_len, dtype: float64

In [112]:
train_texts = set(df_train["content"])
test_texts = set(df_test["content"])
len(train_texts & test_texts)

686

In [113]:
display(df_train[df_train["content"].str.len() > 10000])
display(df_test[df_test["content"].str.len() > 10000])

,content,topic,content_len
3,Фото: Юрий Абрамочкин / РИА Новости Дмитрий Ок...,0,13016
8,Фото: Reza / Getty Images Алевтина Запольская...,3,13258
29,Фото: Xu Jingbo / ZumaPress / Globallookpress....,1,12019
57,Фото: Michael Sohn / AP Максим Коннов Минувший...,1,30325
133,Крушение императорского поезда около станции ...,8,27302
...,...,...,...
18426,"как скоро после постройки тоннель себя окупит,...",6,16336
18458,такое ИЖС и зачем оно нужно?Аббревиатура ИЖС р...,6,17610
18505,"Недвижимость Попов рассказал, насколько проект...",6,17352
18659,Как вы оцениваете сегодняшнюю обстановку на ры...,6,11621


,content
99,Фото: @arman__dikiy Владимир Седов В Казахстан...
227,Фото: Vladimir Sukhachev / Shutterstock / Foto...
307,ПоделитьсяПервая серияВторая серия3.1. ЗМЕЕВО ...
592,Фото: предоставлено пресс-службой ГК «Евростро...
872,"О серии из семи побед, опасных травмах и корру..."
...,...
24462,"Минск, 23 марта 2006 года. Фото: Василий Федо..."
24543,Фото: Георгий Зельма / РИА Новости Максим Семе...
24973,Фото: Alexei Chernyshev / Reuters Александр Не...
25377,Анатолий Балушкин Фото: Из личного архива Анат...


In [114]:
common_texts = list(train_texts & test_texts)[:5]
for t in common_texts:
    print("====")
    print(t[:500])

====
Фото: JD Mason / Unsplash Дарья Коршунова Фитнес-тренер Светлана Бутова назвала виды уличной активности, которыми не стоит заниматься зимой. Ее слова передает «Спорт-Экспресс». Эксперт призвала новичков отказаться от пробежек. «Из-за низкого уровня подготовленности организм может не справиться с нагрузкой в сочетании с минусовой температурой. Неподготовленная дыхательная система может быть подвержена респираторным заболеваниям», — объяснила она. Также специалист предложила перенести с улицы в за
====
Фото: Евгений Павленко / Коммерсантъ Алан Босиков Президент России Владимир Путин заявил о необходимости повысить долю отечественных лекарств на внутреннем рынке. О важности этого он рассказал во время церемонии открытия новых фармацевтических производств, передает ТАСС. По словам главы государства, открытие новых производств поможет еще больше снизить зависимость от поставок лекарственных препаратов из-за рубежа. «Особенно со стороны тех поставщиков, которые создают для нас немало тр

In [115]:
MAX_LEN = 8000

def clean_text(text: str) -> str:
    text = str(text)

    text = text.lower()

    text = re.sub(r'\s+', ' ', text)

    text = text[:MAX_LEN]

    return text.strip()

df_train["content_clean"] = df_train["content"].apply(clean_text)
df_test["content_clean"] = df_test["content"].apply(clean_text)

In [116]:
X = df_train["content_clean"]
y = df_train["topic"]

tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    max_features=100_000,
)

clf = LinearSVC(C=1.0)

pipe = Pipeline([
    ("tfidf", tfidf),
    ("clf", clf),
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

scores = cross_val_score(
    pipe,
    X,
    y,
    cv=cv,
    scoring="f1_macro",
    n_jobs=-1
)

print("CV scores:", scores)
print("CV mean:", scores.mean(), "±", scores.std())

CV scores: [0.9482768  0.95512339 0.95137466 0.96077899 0.94580821]
CV mean: 0.9522724131685265 ± 0.005273456071038666


In [117]:
X = df_train["content_clean"]
y = df_train["topic"]

tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    max_features=100_000,
)

clf = LinearSVC(C=1.0)

pipe = Pipeline([
    ("tfidf", tfidf),
    ("clf", clf),
])

pipe.fit(X, y)

,steps,"[('tfidf', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [118]:
test_preds = pipe.predict(df_test["content_clean"])

In [119]:
submit = pd.read_csv("base_submission_news.csv")
submit["topic"] = test_preds

submit.to_csv("submission_tfidf_svc.csv", index=False)

In [120]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

configs = [
    dict(ngram_range=(1,2), max_features=100_000),
    dict(ngram_range=(1,3), max_features=150_000),
    dict(ngram_range=(1,2), max_features=None),
]

best_score = -np.inf
best_cfg = None

for cfg in configs:
    print("Config:", cfg)
    tfidf = TfidfVectorizer(
        min_df=3,
        max_df=0.9,
        **cfg
    )
    pipe = Pipeline([
        ("tfidf", tfidf),
        ("clf", LinearSVC(C=1.0)),
    ])
    scores = cross_val_score(pipe, X, y, cv=cv, scoring="f1_macro", n_jobs=-1)
    mean_score = scores.mean()
    print("CV mean:", mean_score, "±", scores.std())
    print()

    if mean_score > best_score:
        best_score = mean_score
        best_cfg = cfg

print("Лучшая конфигурация:", best_cfg, "с CV =", best_score)

Config: {'ngram_range': (1, 2), 'max_features': 100000}
CV mean: 0.9522724131685265 ± 0.005273456071038666

Config: {'ngram_range': (1, 3), 'max_features': 150000}


KeyboardInterrupt: 

In [ ]:
# Строим лучший пайплайн
best_tfidf = TfidfVectorizer(
    min_df=3,
    max_df=0.9,
    **best_cfg
)

best_pipe = Pipeline([
    ("tfidf", best_tfidf),
    ("clf", LinearSVC(C=1.0)),
])

# Обучаем на всём train
best_pipe.fit(X, y)

# Предсказываем на test
test_preds = best_pipe.predict(df_test["content_clean"])

# Формируем сабмит
submit = pd.read_csv("base_submission_news.csv")
submit["topic"] = test_preds
submit.to_csv("submission_tfidf_svc_config.csv", index=False)

In [ ]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    max_features=100_000,
)

clf = LogisticRegression(
    C=5.0,
    class_weight="balanced",
    max_iter=500,
    n_jobs=-1,
    solver="lbfgs",
    multi_class="auto",
)

pipe = Pipeline([
    ("tfidf", tfidf),
    ("clf", clf),
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

scores = cross_val_score(
    pipe,
    X,
    y,
    cv=cv,
    scoring="f1_macro",
    n_jobs=-1
)

print("CV scores:", scores)
print("CV mean:", scores.mean(), "±", scores.std())

In [ ]:
# обучаем на всём train
pipe.fit(df_train["content_clean"], df_train["topic"])

# предсказываем
test_preds = pipe.predict(df_test["content_clean"])

submit = pd.DataFrame({
    "index": range(len(test_preds)),
    "topic": test_preds
})
submit.to_csv("submission_logreg_tfidf.csv", index=False)